In [ ]:
using Distributions, LinearAlgebra, Statistics, StatsFuns
cd("../src/")
include("hcmvn.jl")
include("hchol.jl")
include("generate.jl")

In [ ]:
iters = 20
# ns = [512, 1024, 2048]
ns = [256, 512, 1024]
m = 64
d = [1, 2, 4]
N = 10^6
β = 0.3